In [1]:
from os import chdir
chdir('./lib')

In [ ]:
pwd

In [2]:
import pandas as pd
#from sqlalchemy import create_engine
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, SGDRegressor, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression, f_classif

In [3]:
from project_5 import connect_to_postgres, load_data_from_database, make_data_dict, general_model, general_transformer

In [4]:
df = load_data_from_database('postgresql://dsi:correct horse battery staple@joshuacook.me:5432')

In [5]:
df.head(2)

,feat_000,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_491,feat_492,feat_493,feat_494,feat_495,feat_496,feat_497,feat_498,feat_499,label
index,,,,,,,,,,,,,,,,,,,,,
0,485,477,537,479,452,471,491,476,475,473,...,481,477,485,511,485,481,479,475,496,-1
1,483,458,460,487,587,475,526,479,485,469,...,478,487,338,513,486,483,492,510,517,-1


In [6]:
data_dict = make_data_dict(df.drop('label', axis=1), df['label'], 0.25, 51)

In [ ]:
#data_dict['X_train']

In [7]:
data_dict.keys()

['X_test',
 'random_state',
 'X_train',
 'y_train',
 'test_size',
 'y',
 'X',
 'y_test']

In [8]:
X = data_dict['X']
y = data_dict['y']

In [ ]:
scaler = 

In [7]:
general_transformer(StandardScaler(), data_dict)

{'X_test': array([[ 1.11528222, -0.53589971,  0.31929211, ..., -0.51684957,
         -0.07424994,  0.45761632],
        [-0.30367585, -0.70409008,  1.07610477, ..., -0.00464154,
         -0.07424994,  0.45761632],
        [ 0.16931017, -1.37685157, -0.66456435, ...,  1.6783277 ,
          0.78974939, -0.23280125],
        ..., 
        [-0.77666188,  0.57415675,  0.19315666, ...,  2.2637083 ,
          1.32974898,  1.91516452],
        [-0.14601385,  0.13686178, -0.41229347, ...,  0.87342936,
          1.05974919, -0.4245839 ],
        [ 0.16931017,  0.13686178, -0.16002258, ..., -1.24857533,
         -0.1282499 , -2.99447151]]),
 'X_train': array([[ 0.32697218,  0.57415675,  2.48882174, ...,  1.09294709,
         -0.74924943,  1.33981654],
        [-1.72263393, -0.70409008, -0.1095684 , ...,  1.53198254,
         -1.28924901,  2.03023411],
        [-0.14601385,  0.03594755,  1.02565059, ...,  0.21487618,
          0.27674979, -1.80541904],
        ..., 
        [-0.14601385,  0.271414

In [ ]:
scaler['X_train']

In [ ]:
data_dict['y_train']

In [35]:
def general_transformer_standard_workflow(transformer, X, y, 
                                          random_state=None, 
                                          X_test=None, 
                                          y_test=None):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    transformer.fit(X_train, y_train)
    
    X_train = transformer.transform(X_train)
    X_test = transformer.transform(X_test)
#     train_score = transformer.score(X_train, y_train)
#     test_score = transformer.score(X_test, y_test)
    
    return {
        'transformer' : transformer, 
        'X_train'     : X_train,
        'X_test'      : X_test,
        'y_train'     : y_train,
        'y_test'      : y_test,

    }

In [36]:
sk_output = general_transformer_standard_workflow(SelectKBest(f_classif, k=5),X,y)

In [37]:
sk_output['transformer']

SelectKBest(k=5, score_func=<function f_classif at 0x110d009b0>)

In [9]:
selectK_output = general_transformer(SelectKBest(f_classif, k=5), data_dict)

In [10]:
selectK_output.keys()

['X_test',
 'processes',
 'random_state',
 'X_train',
 'y_train',
 'test_size',
 'y',
 'X',
 'y_test']

In [9]:
print selectK_output['model']
print selectK_output['test_score']
print selectK_output['train_score']

KeyError: 'model'

In [ ]:
sele

In [ ]:
general_model(LogisticRegression(), data_dict)

In [61]:
log_model = general_model(LogisticRegression(C=1.0, penalty='l2'), data_dict)

In [62]:
log_model.keys()

['X_test',
 'test_score',
 'x_data',
 'y_train',
 'y_test',
 'X_train',
 'y_data',
 'train_score',
 'random_state',
 'test_size',
 'model']

In [18]:
print log_model['model']
print log_model['test_score']
print log_model['train_score']

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.548484848485
0.813432835821


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [16]:
kbest = SelectKBest(f_classif)
pipeline = Pipeline([('kbest', kbest), ('lr', LogisticRegression())])
grid_search = GridSearchCV(pipeline, {'kbest__k': [1,5,10,50,100]})
grid_search.fit(data_dict['X'], data_dict['y'])

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('kbest', SelectKBest(k=10, score_func=<function f_classif at 0x110d0b9b0>)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'kbest__k': [1, 5, 10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [17]:
grid_search.best_score_

0.61499999999999999

In [31]:
grid_search.score(X,y)

0.61499999999999999

In [36]:
grid_search.predict_proba(X)

array([[ 0.61313045,  0.38686955],
       [ 0.41343645,  0.58656355],
       [ 0.54247906,  0.45752094],
       ..., 
       [ 0.56547143,  0.43452857],
       [ 0.63114618,  0.36885382],
       [ 0.39891217,  0.60108783]])

In [37]:
grid_search.predict_log_proba(X)

array([[-0.48917756, -0.94966772],
       [-0.88325147, -0.53347426],
       [-0.61160579, -0.78193263],
       ..., 
       [-0.5700955 , -0.83349359],
       [-0.46021779, -0.99735485],
       [-0.919014  , -0.50901422]])